In [29]:
import sys
import json
import base64
import numpy as np
import os
from PIL import Image
import io
import os
import tensorflow as tf
# from tf import keras
from tensorflow.keras.models import load_model

ModuleNotFoundError: No module named 'tensorflow.keras'

In [28]:
!pip install -U -q segmentation-models
# !pip install -q numpy==1.18.5
# !pip install -q tensorflow==2.15.0
!pip install -q tensorflow==2.15.0

!pip install -q keras==2.15.0
# !pip install -q keras
!pip install -q tensorflow-estimator



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [23]:

def crop_center(img, cropx, cropy):
    # Assuming img has shape (slices, height, width, channels)
    d, y, x, c = img.shape
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)
    
    # Ensure the cropped image has the same number of channels
    return img[:, starty:starty+cropy, startx:startx+cropx, :]

def pad_slices(scan, target_slices= 50):
    # Get the current number of slices
    current_slices = scan.shape[0]
    # Check if padding is necessary
    if current_slices < target_slices:
        # Calculate padding amounts
        pad_before = (target_slices - current_slices) // 2
        pad_after = target_slices - current_slices - pad_before
        # Pad the scan with zeros on the slices axis (axis 0)
        padded_scan = np.pad(scan, pad_width=((pad_before, pad_after), (0, 0), (0, 0)), mode='constant', constant_values=0)
    elif current_slices > target_slices:
        # Calculate the cropping needed
        start = (current_slices - target_slices) // 2
        end = start + target_slices
        # Crop the scan to the target size
        padded_scan = scan[start:end, :, :]
    else:
        # If the number of slices is already equal to the target, no action is needed
        padded_scan = scan
    return padded_scan


In [22]:
im = []
image_array = np.load('uploads/data_a.npy')
image_array_s = np.load('uploads/data_s.npy')
image_array_c = np.load('uploads/data_c.npy')

for i in range(len(image_array)):
    im.append(Image.fromarray(image_array[i].astype('uint8')))
for i in range(len(image_array_s)):
    im.append(Image.fromarray(image_array_s[i].astype('uint8')))
for i in range(len(image_array_c)):
    im.append(Image.fromarray(image_array_c[i].astype('uint8')))


In [24]:
array_a = pad_slices(image_array)
array_s = pad_slices(image_array_s)
array_c = pad_slices(image_array_c)


all = np.stack((array_a, array_s, array_c), axis=-1)
all = np.array([crop_center(all, 160, 160) ])

In [26]:
model_path = os.path.join('models', 'imageclassifier5.h5')
model = load_model(model_path)
prediction = model.predict(all)


NameError: name 'load_model' is not defined